In [ ]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from keras import backend as K
from keras.callbacks import History 
from keras.models import load_model

import matplotlib.pyplot as plt

Loading csv data:

In [ ]:
muon_dataframe = pd.read_csv('../MuonPOGAnalysisTemplate/output/bxcut_full_3.csv')
muon_dataframe_test = pd.read_csv('../MuonPOGAnalysisTemplate/output/bxcut_org_test.csv')

Use only the 4 primitive case (if the entire dataset is required skip this)

In [ ]:
muon_dataframe = muon_dataframe[muon_dataframe.n_Primitive == 4.0]
muon_dataframe_test = muon_dataframe_test[muon_dataframe_test.n_Primitive == 4.0]
#muon_dataframe = muon_dataframe[muon_dataframe.n_Primitive != 6.0]
#muon_dataframe_test = muon_dataframe_test[muon_dataframe_test.n_Primitive != 6.0]
#muon_dataframe = muon_dataframe[muon_dataframe['genParticle.pt'] < 150]
#muon_dataframe_test = muon_dataframe_test[muon_dataframe_test['genParticle.pt'] < 150]

Function for the feature selection

In [ ]:
def preprocess_features(muon_dataframe):
  """Prepares input features from Muon data set.

  Args:
    muon_dataframe: A Pandas DataFrame expected to contain data
      from muon simulations
  Returns:
    A DataFrame that contains the features to be used for the model.
  """
  selected_features = muon_dataframe[
[#'Event',
 #'n_Primitive',
 '1dtPrimitive.id_r',
 '2dtPrimitive.id_r',
 '3dtPrimitive.id_r',
 '4dtPrimitive.id_r',
 '1dtPrimitive.id_eta',
 '2dtPrimitive.id_eta',
 '3dtPrimitive.id_eta',
 '4dtPrimitive.id_eta',
 '1dtPrimitive.id_phi',
 '2dtPrimitive.id_phi',
 '3dtPrimitive.id_phi',
 '4dtPrimitive.id_phi',
 '1dtPrimitive.phiB',
 '2dtPrimitive.phiB',
 '3dtPrimitive.phiB',
 '4dtPrimitive.phiB',
 '1dtPrimitive.quality',
 '2dtPrimitive.quality',
 '3dtPrimitive.quality',
 '4dtPrimitive.quality',
 'delta_phi12',
 'delta_phi13',
 'delta_phi14',
 'delta_phi23',
 'delta_phi24',
 'delta_phi34'
  ]]
  processed_features = selected_features.copy()
  return processed_features.astype(np.float32)



In [ ]:
def preprocess_targets(muon_dataframe):
  """
  Prepares target features (i.e., labels) from muon data set.

  Args:
    muon_dataframe: A Pandas DataFrame expected to contain data
      from the Muon data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  output_targets = pd.DataFrame()
  output_targets["genParticle.pt"] = muon_dataframe["genParticle.pt"]/200
  return output_targets.astype(np.float32)

In [ ]:
X = preprocess_features(muon_dataframe)
X_test = preprocess_features(muon_dataframe_test)

In [ ]:
Y = preprocess_targets(muon_dataframe)
Y_test = preprocess_targets(muon_dataframe_test)

Filter in quality (grouping between 0 and 1)

In [ ]:
X.loc[X["1dtPrimitive.quality"] < 4, '1dtPrimitive.quality'] = 0.0
X.loc[X["1dtPrimitive.quality"] >= 4, '1dtPrimitive.quality'] = 1.0
X.loc[X["2dtPrimitive.quality"] < 4, '2dtPrimitive.quality'] = 0.0
X.loc[X["2dtPrimitive.quality"] >= 4, '2dtPrimitive.quality'] = 1.0
X.loc[X["3dtPrimitive.quality"] < 4, '3dtPrimitive.quality'] = 0.0
X.loc[X["3dtPrimitive.quality"] >= 4, '3dtPrimitive.quality'] = 1.0
X.loc[X["4dtPrimitive.quality"] < 4, '4dtPrimitive.quality'] = 0.0
X.loc[X["4dtPrimitive.quality"] >= 4, '4dtPrimitive.quality'] = 1.0

X_test.loc[X_test["1dtPrimitive.quality"] < 4, '1dtPrimitive.quality'] = 0.0
X_test.loc[X_test["1dtPrimitive.quality"] >= 4, '1dtPrimitive.quality'] = 1.0
X_test.loc[X_test["2dtPrimitive.quality"] < 4, '2dtPrimitive.quality'] = 0.0
X_test.loc[X_test["2dtPrimitive.quality"] >= 4, '2dtPrimitive.quality'] = 1.0
X_test.loc[X_test["3dtPrimitive.quality"] < 4, '3dtPrimitive.quality'] = 0.0
X_test.loc[X_test["3dtPrimitive.quality"] >= 4, '3dtPrimitive.quality'] = 1.0
X_test.loc[X_test["4dtPrimitive.quality"] < 4, '4dtPrimitive.quality'] = 0.0
X_test.loc[X_test["4dtPrimitive.quality"] >= 4, '4dtPrimitive.quality'] = 1.0

In [ ]:
X.loc[X["1dtPrimitive.id_r"] != 0, '1dtPrimitive.id_r'] = 1.0
X.loc[X["2dtPrimitive.id_r"] != 0, '2dtPrimitive.id_r'] = 1.0
X.loc[X["3dtPrimitive.id_r"] != 0, '3dtPrimitive.id_r'] = 1.0
X.loc[X["4dtPrimitive.id_r"] != 0, '4dtPrimitive.id_r'] = 1.0

In [ ]:
def relu_advanced(x):
    return K.relu(x, max_value=1)

Definition of the keras neural network model:

In [ ]:
def baseline_model(size,epochs,optimizer,X_training,y_training,X_validation,y_validation,output_name):
    # create model
    name="RMSE"
    history = History()
    model = Sequential()
    model.add(Dense(1000, input_dim=26, kernel_initializer='random_normal', activation='sigmoid'))
    model.add(Dropout(rate=0.1))
    model.add(Dense(50, activation='sigmoid'))
    model.add(Dropout(rate=0.1))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    model.fit(x_train, y_train,
          batch_size=size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_validation, y_validation),callbacks=[history])
    predictions = model.predict(X_validation)
    lin_mse = mean_squared_error(y_validation, predictions)
    lin_rmse = np.sqrt(lin_mse)
    msg = "%s: %f" % (name, lin_rmse)
    print(msg)
    fig,ax = plt.subplots()
    ax.scatter(y_validation, predictions, edgecolors=(0, 0, 0))
    ax.set_title('Regression model predictions (validation set)')
    ax.set_xlabel('Measured $p_T$ (GeV/c)')
    ax.set_ylabel('Predicted $p_T$ (GeV/c)')
    ax.plot([Y.min(), Y.max()], [Y.min(), Y.max()], 'k--', lw=4)
    plt.rc('font', size=20)
    plt.rc('axes', titlesize=15)
    plt.rc('axes', labelsize=15)    
    plt.rc('xtick', labelsize=15)   
    plt.rc('ytick', labelsize=15)  
    plt.rc('legend', fontsize=15)    
    plt.rc('figure', titlesize=15)
    plt.tight_layout()
    plt.savefig('1'+ output_name,format='png',dpi=800)
    fig2,ax2 = plt.subplots()
    ax2.plot(history.history['loss'], label='loss')
    ax2.plot(history.history['val_loss'], label='val_loss')
    ax2.set_title('Training and Validation loss per epoch')
    ax2.set_xlabel('# Epoch')
    ax2.set_ylabel('loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig('2'+ output_name,format='png',dpi=800)
    #plt.show()
    del ax,ax2
    return model

Using train/valid split:

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.1)

Model training:

In [ ]:
model = baseline_model(300,90,'Adamax',x_train, y_train, x_valid, y_valid,'Adamax.png')
model.save('my_model_all1.h5')

In [ ]:
model = baseline_model(300,90,'Adagrad',x_train, y_train, x_valid, y_valid,'Adagrad.png')
model.save('my_model_all2.h5')

In [ ]:
model = baseline_model(300,90,'RMSprop',x_train, y_train, x_valid, y_valid,'RMSProp.png')
model.save('my_model_all3.h5')

In [ ]:
model = baseline_model(300,90,'Adam',x_train, y_train, x_valid, y_valid,'Adam.png')
model.save('my_model_all4.h5')

Model testing with indipendent dataset:

In [ ]:
name="RMSE"
model = load_model('my_model.h5')
predictions = model.predict(X_test)
lin_mse = mean_squared_error(Y_test, predictions)
lin_rmse = np.sqrt(lin_mse)
msg = "%s: %f" % (name, lin_rmse)
print(msg)
fig,ax = plt.subplots()
ax.scatter(Y_test*200, predictions*200, edgecolors=(0, 0, 0))
ax.set_title('Regression model predictions (test)')
ax.set_xlabel('Measured $p_{T}$ (GeV/c)')
ax.set_ylabel('Predicted $p_{T}$ (GeV/c)')
ax.plot([Y.min()*200, Y.max()*200], [Y.min()*200, Y.max()*200], 'k--', lw=4)
plt.rc('font', size=13)
plt.rc('axes', titlesize=15)
plt.rc('axes', labelsize=15)    
plt.rc('xtick', labelsize=15)   
plt.rc('ytick', labelsize=15)  
plt.rc('legend', fontsize=15)    
plt.rc('figure', titlesize=15)
plt.tight_layout()
plt.savefig('validation.png',format='png',dpi=800)